<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CPU-Virtualization" data-toc-modified-id="CPU-Virtualization-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CPU Virtualization</a></span></li><li><span><a href="#Memory-Virtualization" data-toc-modified-id="Memory-Virtualization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Memory Virtualization</a></span><ul class="toc-item"><li><span><a href="#抽象的地址空间-(Abstraction-Address-Space)" data-toc-modified-id="抽象的地址空间-(Abstraction-Address-Space)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>抽象的地址空间 (Abstraction Address Space)</a></span><ul class="toc-item"><li><span><a href="#问题？" data-toc-modified-id="问题？-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>问题？</a></span></li><li><span><a href="#基础" data-toc-modified-id="基础-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>基础</a></span></li><li><span><a href="#命令" data-toc-modified-id="命令-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>命令</a></span></li></ul></li><li><span><a href="#内存操作API" data-toc-modified-id="内存操作API-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>内存操作API</a></span></li><li><span><a href="#地址转换（Address-Translation）" data-toc-modified-id="地址转换（Address-Translation）-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>地址转换（Address Translation）</a></span><ul class="toc-item"><li><span><a href="#问题" data-toc-modified-id="问题-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>问题</a></span></li><li><span><a href="#基础" data-toc-modified-id="基础-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>基础</a></span></li></ul></li><li><span><a href="#分段(segmentation)" data-toc-modified-id="分段(segmentation)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>分段(segmentation)</a></span><ul class="toc-item"><li><span><a href="#思考？" data-toc-modified-id="思考？-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>思考？</a></span></li></ul></li></ul></li></ul></div>

Operating System Three Easy Pieces

## CPU Virtualization

两种CPU模式
- 特权模式(privileged mode)，也叫内核模式(kernel mode)
- 用户模式(user mode)




## Memory Virtualization

### 抽象的地址空间 (Abstraction Address Space)

#### 问题？
1. 如何管理可用空间？
2. 空间不足时哪些页面该释放？


#### 基础
1. 概念
   - 操作系统需要提供一个易于使用的物理内存抽象，叫做地址空间（正在运行的程序在系统中的内存视图）。
   - 一个进程的地址空间包含运行程序的所有状态。


2. 虚拟化内存的三个作用
   - 透明(transparency): 操作系统提供的假象不应该被应用程序看破。程序不应该感知到内存被虚拟化，让它自认为拥有自己的私有物理内存。
   - 效率(efficiency): 在时间和空间上提高效率
   - 保护(protection): 确保进程受到保护，不受其它进程的影响，操作系统本身也不受进程影响。保护能够在进程之间提供隔离(isolation)，使之每个进程都能在自己独立的环境中运行，避免出错或恶意进程的影响。


3. **注意** 
   - 虚拟地址只提供地址如何在内存中分布的假象，只有操作系统（和硬件）才知道物理地址。
   - 从用户的角度（程序员）看到内存中的所有地址都是**虚拟地址**


4. 总结
   - 虚拟内存系统为程序提供一个巨大的、稀疏的、私有的地址空间的假象，保存了程序的所有指令和数据。
   - 操作系统在专门硬件的帮助下，通过每一个虚拟内存的索引，将其转化为物理地址，物理内存根据获得的物理地址去获取所需的信息。 
   
#### 命令
- free: 显示系统中的可用和已用内存量
- pmap: 查看进程的内存映像信息
> 参数 解释
  - Address：进程所占的地址空间
  - Kbytes：该虚拟段的大小
  - RSS：设备号（主设备：次设备）
  - Anon：设备的节点号，0表示没有节点与内存相对应
  - Locked：是否允许swapped
  - Mode 权限：r=read, w=write, x=execute, s=shared, p=private(copy on write)
  - Mapping：bash 对应的映像文件名 
  - Resident ：表示在内存中驻留的段的空间   
  - shared ：表示这些北分配的内存是被系统中其他进程共享的。    
  - private ：表示只能被该进程使用的空间大小。你可以发现share的空间不具有 private的属性。
  - Prstat －LP 的输出的意义是：
  - size：就是该进程占用的地址空间。
  - RSS：实际被分配的内存的大小
- 查看进程PID
  - pgep
  - ps aux | grep ...   
   

### 内存操作API

1. 动态分配内存常见误区
   - 忘记分配内存（Forgetting To Allocate Memory）
   ```
    char *src = "hello";
    char *dst; // oops! unallocated
    strcpy(dst, src); // segfault and die
   ```
   - 没有足够的内存，也称为缓冲溢出
   ```
    为字符串声明空间：采用 malloc(strlen(src) + 1) 用法，一边为字符串结束符留出空间。
    char *src = "hello";
    char *dst = (char *) malloc(strlen(src) + 1);
    strcpy(dst, src); // work properly
   ```
   - 忘记初始化分配的内存。忘记在新申请的数据类型中填充一些值，导致程序最终会遇到未初始化的读取，从堆中读取一些未知的数据。
        
   - 忘记释放内存，即内存泄露（memory leak）。如果仍然拥有对某块内存的引用，那么垃圾收集器就不会释放它。
   - 在用完之前释放内存，这种错误称为悬挂指针（dangling pointer）。可能会导致程序崩溃或者覆盖有效的内存。
   - 反复释放内存（Freeing Memory Repeatedly），也被称为重复释放（double free）。导致结果未定义。
   - 错误的调用free()。



2. 查看内存泄露工具
   - Purify：现在是商业产品
   - valgrind：开源工具。（参考：[应用 Valgrind 发现 Linux 程序的内存问题](https://www.ibm.com/developerworks/cn/linux/l-cn-valgrind/index.html)）


3. 底层操作系统支持
   - malloc()与free()函数不是系统调用，而是库调用。malloc库管理虚拟地址空间内的空间。
   - malloc()与free()基于**brk**或**sbrk**系统调用之上。**brk**作用：改变程序分段的位置，堆结束的位置。
   - 调用mmap()，从操作系统获取内存。mmap()在程序中创建一个匿名内存区域，这个区域不与任何特定文件相关联，而是与交换空间（swap space）相关联。


### 地址转换（Address Translation）

#### 问题
1. 怎样在内存中重定位运行的进程，同时对该进程透明（tansparent）?
2. 怎样提供一种虚拟地址从零开始的假象，而实际上地址空间位于另外某个物理地址？


#### 基础
1. 实现CPU虚拟化遵循的准则：受限直接访问（limited direct execution, LDE）。
> 原理：让程序运行的大部分指令直接访问硬件，只在一些关键点（如何发起系统调用或发生时钟中断）由操作系统干预，确保“在正确的时间，正确的地点，做正确的事”。


2. 如何高效、灵活地虚拟化内存？
   - 采用基于硬件的地址转换（hardware-based address translation），简称地址转换。
   - 灵活性：程序能以任何的方式去访问自己的地址空间，让系统更容易编程。
   - 高效性：利用硬件的支持，快速的将所有内存访问操作中的虚拟地址转化为物理地址。
 
 
3. 地址转换
   - 硬件对每次内存访问进行处理（eg: an instruction fetch, load, or store），将指令中的虚拟地址转化为数据实际存储的物理地址。因此，每次内存引用时，硬件都会进行地址转换，将应用程序的内存引用重定向到内存中的实际位置。
   - 虚拟地址：进程自己看到的地址。
   - 物理地址：实际中内存的地址。


4. 动态重定位（dynamic reloction）也称为基址加界限机制（base and bound）
   - CPU需要两个硬件寄存器：基址寄存器和界限寄存器。 
     - 基址寄存器: 将虚拟地址转化为物理地址。
     - 界限寄存器也叫限制寄存器(limit register)：确保虚拟地址在进程地址空间的范围内。
       - 界限寄存器记录地址空间的大小，硬件在将虚拟地址与基址寄存器内容求和之前，需检查这个界限。
       - 界限寄存器中记录地址空间结束的物理地址，硬件在转化虚拟地址到物理地址之后才去检查这个界限。
     - 基址寄存器与界限寄存器组成的结构称为内存管理单元(memory management unit, MMU)，只有在内核模式下才能修改这两个寄存器。
   - 操作系统中有记录哪些空闲物理内存没有使用的**空闲列表(free list)**，以便能够为其它进程分配内存。    
   - 进程产生的所有内存引用，怎样转化为物理地址？
   
   ```
   virtual address也可叫偏移量(offset)。

   physical address = virtual address + base
   ```

   > 采用静态重定位（static reloction）的缺点
   - 不提供访问保护，进程中的错误地址可能导致对其他进程或操作系统内存的非法访问。
   - 一旦完成，很难将内存空间**重定位**到其它位置。  
  
  - 动态重定位的缺点：会造成内存块中大量的空间被浪费，即称为内部碎片。
    > 内部碎片(internal fragmentation): 指已经分配的内存单元内部有未使用的空间，造成了浪费。

### 分段(segmentation)

#### 思考？
1. 为什么要使用分段？

  使用动态重定向，导致在堆和栈之间有一大块“空闲”空间没有被进程使用，但却依然占据了物理内存空间，致使了内存浪费。另外，如果剩余的的物理内存空间无法提供连续区域去放置完整的地址空间（进程的地址空间），那么进程变无法运行。因此，需要支持更大的地址空间，便有了分段的概念。


2. 概念
   - 段：只是地址空间里的一个连续定长的区域。
   - 稀疏地址空间(sparse address spaces)：内存中大量未使用的地址空间。
   - 段错误(segmentation fault)：在支持分段的机器上发生非法的内存访问。也称为段违规(segmentation violation) 。
   - 保护位(protection bit): 为每个段增加了几个位，标识程序是否能够读写该段，或者能否执行该段的代码。
   - 细粒度(fine-grained): 将地址空间划分为大量较小的块。
   - 粗粒度(coarse-grained): 将地址空间分成较大的、粗粒的块。
   - 外部碎片(external fragmentation): 物理内存中充满了许多空闲空间的小洞，很难分配给新的段。


3. 硬件在地址转换是使用段寄存器，是如何知道段内偏移量？虚拟地址引用了哪几个段？
   - 显式方法(explicit approach): 用虚拟地址的开头几位来标识不同的段。![](./figures/标识不同的段.png)
   前两位告诉硬件引用了哪个段，后12位表示段内偏移。
   - 隐式方法( implicit approach)：硬件通过地址产生的方式来确定段。例如：地址由程序计数器产生，那么地址在代码段。


4. 分段的基本原理：系统运行时，地址空间中的不同段被重定位到物理空间中。即堆和栈之间没有使用的区域就不需要分配物理内存，从而能够将更多的地址空间放进物理地址。


5. 操作系统在上下文切换时应该做什么？
   - 各个段寄存器中的内容必须保存和恢复。
   - 管理物理内存的空闲空间。
　
 
6.　使用分段的方式会产生外部碎片，怎样去解决这个难题？
   - 方案一：紧凑物理内存(compact physical memory)，重新安排原有的段。但是，使内存紧凑的成本很高，拷贝段是内存密集型的，会占用大量的处理器时间。
   - 方案二：利用空闲列表管理算法(free-list management algorithm)。
     - 最优匹配(best-fit)
     - 最坏匹配(worst-fit)
     - 伙伴算法(buddy algorithm)
 
  上面采取的两种方案都无法完全消除外部碎片，只是尝试减小外部碎片。最好的方案是：**永远不要分配不同大小的内存块。**


7. 分段的优缺点
   - 优点
     - 要求的算法很容易
     - 很适合硬件完成，地址转换的开销小。
     - 代码共享。
   - 缺点
     - 产生外部碎片
     - 不足以支持更一般化的稀疏空间。